# SACLA Data Processing Notebook (without timetool)
## Overview

This notebook is intended for use during x-ray pump-probe experiments using the SACLA free-electron laser at the SPring-8 synchrotron facility. Measurements are performed by recording the detector images and x-ray intensity monitor values on an event-by-event basis (where an event is a measurement corresponding to a single pump-probe pair) while varying an experimental parameter (typically an angle or delay stage motor). The code below applies several filters to the data, sorts the data into bins according to the scan motor value, and saves the result in the directory `cube_dir` as H5 "cube" files with the following datasets:

- `scan_var` : the value of the scan motor corresponding to each bin
- `imgs` : the sum of all detector images of events falling into each bin
- `i0` : the sum of all x-ray intensity monitor values of events falling into each bin
- `bin_counts` : the number of events falling into each bin

The bin values in `scan_var` are the unique scan motor values over all events in the run.

The binned data is saved separately into "on" and "off" files according to whether the pump shutter was open or closed. For time scans this means that the experimental conditions for all events that get sorted into the "off" file are equivalent regardless of bin.

This notebook does not make use of the spectroencoding diagnostic "timetool" that corrects for jitter between pump and probe pulses. As such it is most appropriate for angle scans, although it can also be used to generate uncorrected time scan cubes.

In the filtering step prior to binning, the following filters are applied:
- `nan_filt` : filters out all events where either the timetool correction or intensity monitor values were `NaN`
- `intens_filt` : filters out all events with intensity monitor values falling outside the range specified in `intens_thresh`
- `x_filt` : filters out all events with closed or invalid x-ray shutter status
- `las_filt` : filters out all events with invalid pump shutter status

The detector images must be subtracted by a "dark" background image of the detector before they are sorted into bins. These background files are created by acquiring detector images with both the pump and probe shutters closed and using the `make_bg.py` script to save the resulting data as an H5 file. `bg_num` denotes the run number corresponding to the background measurement, and `bg_dir` is the directory in which the background file is stored.

Motor values, detector images, and intensity monitor values are obtained using SACLA's `dbpy` and `stpy` python modules. Motor values are reported in pulses, which need to be converted to appropriate units with the conversion factor `pulse_to_val`.

A python script version of this code is provided in `cube_no-timetool.py`, which can be run at the command line or submitted to the cluster using `qsub`.

## Code
### Import modules:

In [1]:
import sys

DataAccessUserAPI_path = '/home/software/SACLA_tool/DataAccessUserAPI/latest/python/lib'
SACLA_python_modules = '/home/software/opt/intel/oneapi/intelpython/python3.7/lib/python3.7/site-packages'

if not DataAccessUserAPI_path in sys.path:
    sys.path.insert(0, DataAccessUserAPI_path)
if not SACLA_python_modules in sys.path:
    sys.path.insert(0, SACLA_python_modules)

import numpy as np
import dbpy
import stpy
import h5py
from time import time

### Definitions (may need to change these):

In [4]:
# Experiment parameters
hi_tag = 202301
bl = 3 # specify beamline
run = 1293071 # Run number

# Motor names and filter names -- before the experiment check that these are correct
det_ID = 'MPCCD-1N0-M06-002'
xon_name = 'xfel_bl_3_shutter_1_open_valid/status'
xoff_name = 'xfel_bl_3_shutter_1_close_valid/status'
lason_name = 'xfel_bl_3_lh1_shutter_1_open_valid/status'
lasoff_name = 'xfel_bl_3_lh1_shutter_1_close_valid/status'
delay_name = 'xfel_bl_3_st_2_motor_1/position'
phi_name = 'xfel_bl_3_st_2_motor_35/position'
chi_name = 'xfel_bl_3_st_2_motor_53/position'
intens_6_name = 'xfel_bl_3_st_2_pd_user_6_fitting_peak/voltage'
intens_7_name = 'xfel_bl_3_st_2_pd_user_7_fitting_peak/voltage'
intens_name = intens_6_name

cube_dir = '/work/raduncan/2023A8060_work/raduncan/cubes_post/' # Directory where cubes are saved


# Background information
bg_num = 1292973 # Run number of background scan
bg_dir = '/work/raduncan/2023A8060_work/backgrounds/' # Directory where background files are saved

# The motor that is being scanned -- make sure this is right!
scan_motor_name = phi_name

# Conversion factors
pulse_to_val = 0.0001 # For phi
# pulse_to_val = 0.001 # For chi
#pulse_to_val = 0.006671 # For delay

# Threshold values
intens_thresh = (-np.inf, np.inf) # Filter for i0 values
energy_thresh = 2100 # Energy threshold for pixels -- set all pixels below this value to zero

# Binning parameters -- these need to be manually set for each run
binval_min = -212.36
binval_max = -210.86
num_bins = 151
bin_delta = (binval_max - binval_min)/(num_bins - 1)
scan_var = np.linspace(binval_min, binval_max, num_bins)

### Filter and bin the scan:

In [5]:
start_time = time()

print('Loading scan data...')

# Load non-detector data
taglist = dbpy.read_taglist_bydetid(bl, run, det_ID)
nptaglist = np.array(taglist)
num_shots = len(taglist)
intens = np.array(dbpy.read_syncdatalist_float(intens_6_name, hi_tag, taglist)) + np.array(dbpy.read_syncdatalist_float(intens_7_name, hi_tag, taglist))
intens0 = intens
lason = np.array(dbpy.read_syncdatalist_float(lason_name, hi_tag, taglist), dtype=bool)
lasoff = np.array(dbpy.read_syncdatalist_float(lasoff_name, hi_tag, taglist), dtype=bool)
xon = np.array(dbpy.read_syncdatalist_float(xon_name, hi_tag, taglist), dtype=bool)
xoff = np.array(dbpy.read_syncdatalist_float(xoff_name, hi_tag, taglist), dtype=bool)
scan_vals = np.array(dbpy.read_syncdatalist_float(scan_motor_name, hi_tag, taglist))*pulse_to_val

# Create filters
nan_filt = np.logical_not(np.isnan(intens))
intens_filt = np.logical_and((intens > intens_thresh[0]), (intens < intens_thresh[-1]))
x_filt = np.logical_and(xon, np.logical_not(xoff))
las_filt = np.logical_or(np.logical_and(lason, np.logical_not(lasoff)), np.logical_and(lasoff, np.logical_not(lason)))
tot_filt = np.logical_and.reduce([nan_filt, intens_filt, x_filt, las_filt])

# Apply filters
intens = intens[tot_filt]
nptaglist = nptaglist[tot_filt]
taglist = tuple(map(int, nptaglist.tolist()))
lason = lason[tot_filt]
lasoff = lasoff[tot_filt]
scan_vals = scan_vals[tot_filt]
# scan_var = np.unique(np.round(scan_vals, 0))
# bin_delta = np.diff(scan_var)[0]
# num_bins = len(scan_var)

# Generate bin indices for events
bin_inds = np.digitize(scan_vals - bin_delta/2, scan_var, right=True)
bin_inds[bin_inds > num_bins - 1] = num_bins - 1
bin_inds[bin_inds < 0] = 0

# Separate events into laser-on and laser-off
on_tags = tuple(map(int, nptaglist[lason].tolist()))
off_tags = tuple(map(int, nptaglist[lasoff].tolist()))
intens_on = intens[lason]
intens_off = intens[lasoff]
scan_vals_on = scan_vals[lason]
scan_vals_off = scan_vals[lasoff]
bin_inds_on = bin_inds[lason]
bin_inds_off = bin_inds[lasoff]

# Initialize detector reading objects and cube fields
reader = stpy.StorageReader(det_ID, bl, (run, run))
buffer = stpy.StorageBuffer(reader)

i0_on = np.zeros(num_bins)
bin_counts_on = np.zeros(num_bins)

i0_off = np.zeros(num_bins)
bin_counts_off = np.zeros(num_bins)

print('Scan motor: ' + scan_motor_name)
print(f'Scan values ranging from {min(scan_vals)} to {max(scan_vals)}')
print(f'Binning into {num_bins} bins from {min(scan_var)} to {max(scan_var)}')
print(f'Bin delta is {bin_delta}')
print(f'{num_shots} shots before filtering')
print(f'{len(on_tags) + len(off_tags)} shots after filtering')
print(f'{len(on_tags)} laser-on shots, {len(off_tags)} laser-off shots')

print('\nBinning laser-on shots...')
for ii,tag in enumerate(on_tags):
    reader.collect(buffer, tag)
    img = buffer.read_det_data(0)
    i0 = intens_on[ii]
    
    
    if ii == 0:
        imgs_on = np.zeros((num_bins, img.shape[0], img.shape[1]))
        imgs_off = np.zeros((num_bins, img.shape[0], img.shape[1]))
        
        # Load background        
        with h5py.File(bg_dir + f'run{bg_num:04d}_{det_ID}.h5', 'r') as f:
            img_bg = f['img'][:]
            count_bg = f['count'][()]
            
        img_bg /= count_bg
        img_bg = np.reshape(img_bg, img.shape)
        
        
    gain = np.copy(buffer.read_det_info(0)['mp_absgain'])
    img -= img_bg
    img *= gain
    
    jj = bin_inds_on[ii]
    img[img < energy_thresh] = 0
    imgs_on[jj,:,:] += img
    i0_on[jj] += i0
    bin_counts_on[jj] += 1
    
    if (ii%100 == 0) and (ii > 0):
        print(f'Binned {ii} shots...')

print('Done binning laser-on shots')

print('\nBinning laser-off shots...')
for ii,tag in enumerate(off_tags):
    reader.collect(buffer, tag)
    img = buffer.read_det_data(0)
    i0 = intens_off[ii]
    
    if ii == 0:
        imgs_off = np.zeros((num_bins, img.shape[0], img.shape[1]))
        if len(on_tags) == 0:
            imgs_on = np.zeros((num_bins, img.shape[0], img.shape[1]))
        
        # Load background
        with h5py.File(bg_dir + f'run{bg_num:04d}_{det_ID}.h5', 'r') as f:
            img_bg = f['img'][:]
            count_bg = f['count'][()]
            
        img_bg /= count_bg
        img_bg = np.reshape(img_bg, img.shape)
    
    gain = np.copy(buffer.read_det_info(0)['mp_absgain'])
    img -= img_bg
    img *= gain
    
    jj = bin_inds_off[ii]
    img[img < energy_thresh] = 0
    imgs_off[jj,:,:] += img
    i0_off[jj] += i0
    bin_counts_off[jj] += 1
    
    if (ii%100 == 0) and (ii > 0):
        print(f'Binned {ii} shots...')

print('Done binning')
end_time = time()
print(f'Took {end_time - start_time} seconds to bin the run.')

Loading scan data...


/home/raduncan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater
/home/raduncan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less


Scan motor: xfel_bl_3_st_2_motor_35/position
Scan values ranging from -213.97 to -212.47
Binning into 151 bins from -212.36 to -210.86
Bin delta is 0.01
13842 shots before filtering
13650 shots after filtering
0 laser-on shots, 13650 laser-off shots

Binning laser-on shots...
Done binning laser-on shots

Binning laser-off shots...
Binned 100 shots...
Binned 200 shots...
Binned 300 shots...
Binned 400 shots...
Binned 500 shots...
Binned 600 shots...
Binned 700 shots...
Binned 800 shots...
Binned 900 shots...
Binned 1000 shots...
Binned 1100 shots...
Binned 1200 shots...
Binned 1300 shots...
Binned 1400 shots...
Binned 1500 shots...
Binned 1600 shots...
Binned 1700 shots...
Binned 1800 shots...
Binned 1900 shots...
Binned 2000 shots...
Binned 2100 shots...
Binned 2200 shots...
Binned 2300 shots...
Binned 2400 shots...
Binned 2500 shots...
Binned 2600 shots...
Binned 2700 shots...
Binned 2800 shots...
Binned 2900 shots...
Binned 3000 shots...
Binned 3100 shots...
Binned 3200 shots...
Binn

### Save the binned data as H5 cubes:

In [7]:
# Save cube to hdf5
cubename_on = f'run{run:04d}_on.h5'
cubename_off = f'run{run:04d}_off.h5'
with h5py.File(cube_dir + cubename_on, 'w') as f:
    f.create_dataset('scan_var', data=scan_var)
    f.create_dataset('imgs', data=imgs_on)
    f.create_dataset('i0', data=i0_on)
    f.create_dataset('bin_counts', data=bin_counts_on)

with h5py.File(cube_dir + cubename_off, 'w') as f:
    f.create_dataset('scan_var', data=scan_var)
    f.create_dataset('imgs', data=imgs_off)
    f.create_dataset('i0', data=i0_off)
    f.create_dataset('bin_counts', data=bin_counts_off)

print(f'Saved cubes for run {run} in {cube_dir}')
print('Done.')

Saved cubes for run 1293071 in /work/raduncan/2023A8060_work/raduncan/cubes_post/
Done.
